In [17]:
import pandas as pd
import re
import numpy as np
print("libaries successfully imported :)")

libaries successfully imported :)


In [18]:
# The data spans from Start (YYYY-MM-DD): 2024-11-04 08:00:00 SGT End: 2025-10-13 07:59:59 SGT to align with the crypto price data 
# UTC 2024-11-04 00:00:00 to 2025-10-12 23:59:59

# We taking data from 2024-11-03 to 2025-10-13 to have a full day before the start date of 2024-11-05 for any rolling calculations

data = pd.read_json('../Data/whale_alert_031124_131025.json')

#Structure of the message data downloaded
message_data = data.get("messages")
message_data[0]

{'id': 82893,
 'type': 'message',
 'date': '2024-11-03T08:26:49',
 'date_unixtime': '1730593609',
 'edited': '2024-11-03T08:26:52',
 'edited_unixtime': '1730593612',
 'from': 'Whale Alert',
 'from_id': 'channel1309043988',
 'text': ['🚨 🚨 🚨 🚨 🚨  45,679 ',
  {'type': 'hashtag', 'text': '#WETH'},
  ' (113,879,074 USD) transferred from unknown wallet to unknown wallet\n',
  {'type': 'text_link',
   'text': 'Details',
   'href': 'https://whale-alert.io/transaction/ethereum/0x3d900dfb58f61b63145cc93d3852c02093e0adfb48b20968d09b09b00f6d19e5'},
  ''],
 'text_entities': [{'type': 'plain', 'text': '🚨 🚨 🚨 🚨 🚨  45,679 '},
  {'type': 'hashtag', 'text': '#WETH'},
  {'type': 'plain',
   'text': ' (113,879,074 USD) transferred from unknown wallet to unknown wallet\n'},
  {'type': 'text_link',
   'text': 'Details',
   'href': 'https://whale-alert.io/transaction/ethereum/0x3d900dfb58f61b63145cc93d3852c02093e0adfb48b20968d09b09b00f6d19e5'},
  {'type': 'plain', 'text': ''}]}

In [19]:
# We will parse the data to extract relevant information used in the studies such as datetime, bitcoin transactions, bitcoin change and usd changes
# Store each message as a dictionary with 'date' and 'text' (joined string),
# but for hashtags, only keep the hashtag value (e.g., BTC)
# Remove emojis, hashtags, double spaces, and newlines from text
# Only include messages where text contains 'transferred' and 'BTC'
flattened_sentiment_df = []
emoji_pattern = re.compile('[\U00010000-\U0010ffff\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+', flags=re.UNICODE)
for msg in message_data:
    unixdatetime = msg.get("date_unixtime")
    text_items = []
    for item in msg.get("text"):
        if isinstance(item, dict):
            # Only keep the hashtag value if type is 'hashtag'
            if item.get('type') == 'hashtag' and 'text' in item:
                # Remove hashtag symbol
                text_items.append(item['text'].replace('#', ''))
        else:
            # Filter out unwanted string markers
            if not (str(item).startswith('type:') or str(item).startswith('text:')):
                text_items.append(item)
    text_str = ' '.join([str(x) for x in text_items])
    # Remove emojis
    text_str = emoji_pattern.sub('', text_str)
    # Remove newlines and double spaces
    text_str = text_str.replace('\n', ' ')
    text_str = re.sub(' +', ' ', text_str).strip()
    # Only include if 'transferred' and 'ETH' are in text (CHANGE FOR OTHER CRYPTO)
    if 'transferred' in text_str and 'ETH' in text_str:
        row_dict = {'unixdatetime': unixdatetime, 'text': text_str}
        flattened_sentiment_df.append(row_dict)
flattened_sentiment_df[0:10]

[{'unixdatetime': '1730593609',
  'text': '45,679 WETH (113,879,074 USD) transferred from unknown wallet to unknown wallet'},
 {'unixdatetime': '1730704234',
  'text': '50,000 ETH (123,833,627 USD) transferred from Arbitrum to unknown wallet'},
 {'unixdatetime': '1730704575',
  'text': '11,200 ETH (27,703,319 USD) transferred from Ceffu to unknown wallet'},
 {'unixdatetime': '1730704619',
  'text': '12,017 ETH (29,718,630 USD) transferred from unknown wallet to Binance'},
 {'unixdatetime': '1730706009',
  'text': '13,216 ETH (32,548,387 USD) transferred from unknown wallet to Coinbase'},
 {'unixdatetime': '1730733439',
  'text': '13,216 ETH (32,339,685 USD) transferred from unknown wallet to Coinbase'},
 {'unixdatetime': '1730924474',
  'text': '13,405 ETH (35,714,357 USD) transferred from unknown wallet to Coinbase'},
 {'unixdatetime': '1730930979',
  'text': '13,441 ETH (36,171,792 USD) transferred from unknown wallet to Coinbase'},
 {'unixdatetime': '1730946709',
  'text': '38,400 E

In [20]:
# we will now format the data such that the information required in the studies is extracted 
# Extract UTC timestamp, Cryptocurrency, From, To, number of coins, and value (USD) for the final dataframe
# Only keep the first text with parentheses containing 'USD' for the Value (USD) field
final_records = []
for row in flattened_sentiment_df:
    unix_ts = row['unixdatetime']
    utc_timestamp = pd.to_datetime(unix_ts, unit='s', utc=True)
    text = row['text']

    # Extract number of coins (first number with comma)
    num_coins_match = re.search(r'(\d{1,3}(?:,\d{3})*(?:\.\d+)?)', text)
    num_coins = num_coins_match.group(1) if num_coins_match else None
    num_coins = pd.to_numeric(num_coins.replace(',', '') if num_coins else None)

    # Only keep the first text with parentheses containing 'USD'
    value_match = re.search(r"\(([^()]*)\)", text)
    value_raw = value_match.group(1) if value_match else None
    value = pd.to_numeric(value_raw.replace('USD', '').replace(',', '').strip())

    # Extract coin (first all-caps word after number of coins)
    coin_match = re.search(r'(\d{1,3}(?:,\d{3})*(?:\.\d+)?)[ ]*([A-Z]{2,10})', text)
    coin = coin_match.group(2) if coin_match else None

    # Extract 'from' and 'to' using regex
    from_match = re.search(r'transferred from (.*?) to', text)
    to_match = re.search(r'to (.*?)(?:\.|$)', text)
    from_entity = from_match.group(1).strip() if from_match else None
    to_entity = to_match.group(1).lstrip() if to_match else None
    final_records.append({
        'UTC timestamp': utc_timestamp,
        'Cryptocurrency': coin,
        'Number of Coins': num_coins,
        'Value (USD)': value,
        'From': from_entity,
        'To': to_entity
    })
final_df = pd.DataFrame(final_records)  # rename
final_df 

/var/folders/sr/m5w47sf11d525m54586gg3c00000gn/T/ipykernel_42448/2503583141.py:7: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  utc_timestamp = pd.to_datetime(unix_ts, unit='s', utc=True)


,UTC timestamp,Cryptocurrency,Number of Coins,Value (USD),From,To
0,2024-11-03 00:26:49+00:00,WETH,45679,113879074,unknown wallet,unknown wallet
1,2024-11-04 07:10:34+00:00,ETH,50000,123833627,Arbitrum,unknown wallet
2,2024-11-04 07:16:15+00:00,ETH,11200,27703319,Ceffu,unknown wallet
3,2024-11-04 07:16:59+00:00,ETH,12017,29718630,unknown wallet,Binance
4,2024-11-04 07:40:09+00:00,ETH,13216,32548387,unknown wallet,Coinbase
...,...,...,...,...,...,...
1584,2025-10-11 15:52:58+00:00,ETH,27159,103923573,unknown wallet,unknown wallet
1585,2025-10-11 20:06:34+00:00,ETH,26336,98816239,FalconX,unknown wallet
1586,2025-10-11 21:07:33+00:00,ETH,14643,53624921,FalconX,unknown wallet
1587,2025-10-11 21:09:38+00:00,ETH,24674,90371113,unknown wallet,Robinhood


In [21]:
# Filter for only ETH transactions
final_df['Cryptocurrency'] == 'ETH'

# Filter for transactions where either 'From' or 'To' contains 'unknown' (case-insensitive), but not both
from_unknown = final_df['From'].str.lower().str.contains('unknown', na=False)
to_unknown = final_df['To'].str.lower().str.contains('unknown', na=False)
# Only keep rows where exactly one of 'From' or 'To' is unknown
unknown_mask = from_unknown ^ to_unknown
unknown_transactions = final_df[unknown_mask]
len(unknown_transactions)


1215

In [22]:
# sepparating exchange to unknown and unknown to exchange transactions into 2 df


exchange_to_unknown = unknown_transactions[unknown_transactions['To'].str.contains('unknown', case=False, na=False)][['Number of Coins', 'Value (USD)', 'UTC timestamp']]
exchange_to_unknown = exchange_to_unknown[['UTC timestamp', 'Value (USD)', 'Number of Coins']]

unknown_to_exchange = unknown_transactions[unknown_transactions['From'].str.contains('unknown', case=False, na=False)][['Number of Coins', 'Value (USD)', 'UTC timestamp']]
unknown_to_exchange = unknown_to_exchange[['UTC timestamp', 'Value (USD)', 'Number of Coins']]



In [23]:
exchange_to_unknown['value_change'] = exchange_to_unknown['Value (USD)']
exchange_to_unknown['coins_change'] = exchange_to_unknown['Number of Coins']

unknown_to_exchange['value_change'] = -unknown_to_exchange['Value (USD)']
unknown_to_exchange['coins_change'] = -unknown_to_exchange['Number of Coins']

In [24]:
df=pd.concat([exchange_to_unknown, unknown_to_exchange], ignore_index=True, sort=False)

df['whale_to_exchange_usd'] = -df['value_change'].where(df['value_change'] < 0, 0)
df['exchange_to_whale_usd'] = df['value_change'].where(df['value_change'] > 0, 0)
df['whale_to_exchange_coins'] = -df['coins_change'].where(df['value_change'] < 0, 0)
df['exchange_to_whale_coins'] = df['coins_change'].where(df['value_change'] > 0, 0)

In [25]:
df['datetimeday'] = pd.to_datetime(df['UTC timestamp'], errors='coerce').dt.date

temp5=pd.to_datetime(df['UTC timestamp'], errors='coerce').dt.floor('1h')
temp5=pd.to_datetime(temp5.map(lambda t: t.strftime('%Y-%m-%d %H:%M')))
df['datetime1h']=temp5

df

,UTC timestamp,Value (USD),Number of Coins,value_change,coins_change,whale_to_exchange_usd,exchange_to_whale_usd,whale_to_exchange_coins,exchange_to_whale_coins,datetimeday,datetime1h
0,2024-11-04 07:10:34+00:00,123833627,50000,123833627,50000,0,123833627,0,50000,2024-11-04,2024-11-04 07:00:00
1,2024-11-04 07:16:15+00:00,27703319,11200,27703319,11200,0,27703319,0,11200,2024-11-04,2024-11-04 07:00:00
2,2024-11-07 17:34:55+00:00,21740476,7591,21740476,7591,0,21740476,0,7591,2024-11-07,2024-11-07 17:00:00
3,2024-11-08 08:14:39+00:00,28606186,9874,28606186,9874,0,28606186,0,9874,2024-11-08,2024-11-08 08:00:00
4,2024-11-08 22:07:39+00:00,29635343,10037,29635343,10037,0,29635343,0,10037,2024-11-08,2024-11-08 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...
1210,2025-10-11 02:46:39+00:00,134805875,35096,-134805875,-35096,134805875,0,35096,0,2025-10-11,2025-10-11 02:00:00
1211,2025-10-11 09:36:01+00:00,106985250,27892,-106985250,-27892,106985250,0,27892,0,2025-10-11,2025-10-11 09:00:00
1212,2025-10-11 12:16:48+00:00,57491370,15000,-57491370,-15000,57491370,0,15000,0,2025-10-11,2025-10-11 12:00:00
1213,2025-10-11 21:09:38+00:00,90371113,24674,-90371113,-24674,90371113,0,24674,0,2025-10-11,2025-10-11 21:00:00


In [26]:
### Whale txn count, whale net usd, whale net usd
hourly_df = (
    df.groupby('datetime1h')
      .agg(
        whale_txn_count=('value_change', 'count'),         
        whale_net_usd=('value_change', 'sum'),            
        wtoe_usd=('exchange_to_whale_usd', 'sum'),   
        etow_usd=('whale_to_exchange_usd', 'sum'),
        wtoe_coins=('exchange_to_whale_coins', 'sum'),
        etow_coins=('whale_to_exchange_coins', 'sum')     
      )
      .reset_index()
)
hourly_df

,datetime1h,whale_txn_count,whale_net_usd,wtoe_usd,etow_usd,wtoe_coins,etow_coins
0,2024-11-04 07:00:00,4,89269929,151536946,62267017,61200,25233
1,2024-11-04 15:00:00,1,-32339685,0,32339685,0,13216
2,2024-11-06 20:00:00,1,-35714357,0,35714357,0,13405
3,2024-11-06 22:00:00,1,-36171792,0,36171792,0,13441
4,2024-11-07 07:00:00,1,-23877104,0,23877104,0,8492
...,...,...,...,...,...,...,...
805,2025-10-11 09:00:00,2,-28497188,78488062,106985250,20500,27892
806,2025-10-11 10:00:00,1,76678254,76678254,0,20000,0
807,2025-10-11 12:00:00,1,-57491370,0,57491370,0,15000
808,2025-10-11 20:00:00,1,98816239,98816239,0,26336,0


In [27]:
# Make hourly index (so rolling-by-rows works) and compute 24h features
hourly_df['datetime1h'] = pd.to_datetime(hourly_df['datetime1h'], utc=True)
hourly_df = hourly_df.set_index('datetime1h').sort_index()

In [28]:
# 24h rolling net USD 
hourly_df['whale_net_usd_24h'] = (
    hourly_df['whale_net_usd']
    .rolling(window=24, min_periods=1)
    .sum()
)


# Whale burst flag using past 24 hours (95th percentile), shifted 1h to avoid look-ahead
hourly_df['whale_burst_threshold'] = (
    hourly_df['whale_txn_count']
    .rolling(window=24, min_periods=0)
    .quantile(0.95)
    .shift(1) # shift to avoid look-ahead, means the window will be stricktly before the current hour
)

hourly_df['whale_burst_flag'] = (
    (hourly_df['whale_txn_count'] >= hourly_df['whale_burst_threshold'])
    .astype('Int64')   # will be <NA> for early hours without enough history
)
hourly_df = hourly_df[hourly_df.index >= '2024-11-04 00:00:00']

hourly_df.drop(columns=['whale_burst_threshold'], inplace=True)


### log and power transformation
for skewness > 1:

if positively skewed: log

if negatively skewed: signed log

if percentage 0-100: use power transform 

check skewness, if skew still > 1 after log:
power transform those features

In [29]:
from scipy.stats import skew
merged_log = hourly_df.copy()
# Select numeric columns
numeric_cols = merged_log.select_dtypes(include=[np.number]).columns

# Compute skewness
skew_vals = merged_log[numeric_cols].apply(lambda x: skew(x.dropna()))
skew_df = pd.DataFrame({"feature": skew_vals.index, "skew": skew_vals.values})
skew_df = skew_df.sort_values(by="skew", ascending=False)

print(skew_df)

             feature      skew
4         wtoe_coins  6.581739
2           wtoe_usd  5.812689
3           etow_usd  4.862738
5         etow_coins  3.488191
0    whale_txn_count  2.952670
7   whale_burst_flag  1.741979
1      whale_net_usd  0.298774
6  whale_net_usd_24h -0.376102


In [30]:
## logging values so that it is not skewed by large transactions
def signed_log1p(x):
    x = np.asarray(x, dtype=float)
    return np.sign(x) * np.log1p(np.abs(x))


In [31]:
# log transform certain features
log_features = ['etow_usd', 'etow_coins','whale_txn_count', 'wtoe_usd', 'wtoe_coins'] # we ignore whale_burst_flag since it is binary

for col in log_features:
    merged_log[f'{col}_log'] = signed_log1p(merged_log[col])
    merged_log = merged_log.drop(columns=[col])





In [32]:
# Compute skewness
numeric_cols = merged_log.select_dtypes(include=[np.number]).columns
skew_vals = merged_log[numeric_cols].apply(lambda x: skew(x.dropna()))
skew_df = pd.DataFrame({"feature": skew_vals.index, "skew": skew_vals.values})
skew_df = skew_df.sort_values(by="skew", ascending=False)

print(skew_df)

               feature      skew
5  whale_txn_count_log  1.765856
2     whale_burst_flag  1.741979
0        whale_net_usd  0.298774
7       wtoe_coins_log  0.146283
6         wtoe_usd_log  0.122273
1    whale_net_usd_24h -0.376102
4       etow_coins_log -0.767836
3         etow_usd_log -0.800929


In [33]:
from sklearn.preprocessing import PowerTransformer
still_skew_col = [
    "whale_txn_count_log",
]

pt = PowerTransformer(method='yeo-johnson')
merged_log[still_skew_col] = pt.fit_transform(merged_log[still_skew_col])

In [34]:
# Compute skewness
numeric_cols = merged_log.select_dtypes(include=[np.number]).columns
skew_vals = merged_log[numeric_cols].apply(lambda x: skew(x.dropna()))
skew_df = pd.DataFrame({"feature": skew_vals.index, "skew": skew_vals.values})
skew_df = skew_df.sort_values(by="skew", ascending=False)
print(skew_df)

               feature      skew
2     whale_burst_flag  1.741979
5  whale_txn_count_log  0.836579
0        whale_net_usd  0.298774
7       wtoe_coins_log  0.146283
6         wtoe_usd_log  0.122273
1    whale_net_usd_24h -0.376102
4       etow_coins_log -0.767836
3         etow_usd_log -0.800929


In [35]:
merged_log.isna().sum()

whale_net_usd          0
whale_net_usd_24h      0
whale_burst_flag       0
etow_usd_log           0
etow_coins_log         0
whale_txn_count_log    0
wtoe_usd_log           0
wtoe_coins_log         0
dtype: int64

In [36]:
merged_log.fillna(0, inplace=True)
merged_log.columns

Index(['whale_net_usd', 'whale_net_usd_24h', 'whale_burst_flag',
       'etow_usd_log', 'etow_coins_log', 'whale_txn_count_log', 'wtoe_usd_log',
       'wtoe_coins_log'],
      dtype='object')

In [37]:
merged_log = merged_log.reset_index(drop=False)
merged_log 

,datetime1h,whale_net_usd,whale_net_usd_24h,whale_burst_flag,etow_usd_log,etow_coins_log,whale_txn_count_log,wtoe_usd_log,wtoe_coins_log
0,2024-11-04 07:00:00+00:00,89269929,89269929.0,0,17.946942,10.135948,1.711019,18.836340,11.021919
1,2024-11-04 15:00:00+00:00,-32339685,56930244.0,0,17.291806,9.489259,-0.669956,0.000000,0.000000
2,2024-11-06 20:00:00+00:00,-35714357,21215887.0,0,17.391063,9.503458,-0.669956,0.000000,0.000000
3,2024-11-06 22:00:00+00:00,-36171792,-14955905.0,0,17.403790,9.506139,-0.669956,0.000000,0.000000
4,2024-11-07 07:00:00+00:00,-23877104,-38833009.0,0,16.988431,9.046998,-0.669956,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
805,2025-10-11 09:00:00+00:00,-28497188,-694288206.0,1,18.488202,10.236131,1.388225,18.178457,9.928229
806,2025-10-11 10:00:00+00:00,76678254,-670071582.0,0,0.000000,0.000000,-0.669956,18.155129,9.903538
807,2025-10-11 12:00:00+00:00,-57491370,-786301705.0,0,17.867145,9.615872,-0.669956,0.000000,0.000000
808,2025-10-11 20:00:00+00:00,98816239,-619236287.0,0,0.000000,0.000000,-0.669956,18.408773,10.178730


In [38]:
merged_log.to_csv('../Data/whale_alert_eth.csv', index=False)